In [1]:
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)

from sklearn import preprocessing
from sklearn.metrics import r2_score, mean_absolute_error
import xgboost as xgb
from joblib import load, dump
from sklearn.model_selection import train_test_split

# make df for web with pred

In [4]:
df = pd.read_csv('1_taipei_purged.csv')

dfp = pd.read_csv('pop.csv')
dfp = dfp[['year', 'district', 'pop', 'sex_ratio', 'in', 'out', 'growth', 'marry_rate']]

dff = pd.read_csv('AllFinancialCols.csv')
dff = dff[['date', 'load_archi', 'load_house', 'M1B', 'income_rate']]

df['district'] = df['district'].str[3:5]
df['year'] = df['date'] // 100

df = pd.merge(df, dfp, on=['year', 'district'])
df = pd.merge(df, dff, on='date')

df = df.drop(['address', 'latitude', 'longitude', 'style', 'percent', 'management', 'district', 'year'], axis=1)
df.loc[:, 'date'] = df.loc[:, 'date'] % 100 + (df.loc[:, 'date'] // 100 - 2013 ) * 12

In [5]:
# train
model = load('house.joblib')

X = df.drop(['total_price', 'avg'], axis=1)
y = df[['total_price']]

scaler = preprocessing.StandardScaler().fit(X)
X = scaler.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

model.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.05, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.25, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=13, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0.3, reg_lambda=0.09, ...)

In [6]:
# get all predict 
y_pred = model.predict(X)

pred = pd.DataFrame(y_pred, columns = ['pred'])

In [7]:
# reload df
df = pd.read_csv('1_taipei_purged.csv')

dfp = pd.read_csv('pop.csv')
dfp = dfp[['year', 'district', 'pop', 'sex_ratio', 'in', 'out', 'growth', 'marry_rate']]

dff = pd.read_csv('AllFinancialCols.csv')
dff = dff[['date', 'load_archi', 'load_house', 'M1B', 'income_rate']]

df['district'] = df['district'].str[3:5]
df['year'] = df['date'] // 100

df = pd.merge(df, dfp, on=['year', 'district'])
df = pd.merge(df, dff, on='date')

In [9]:
# join pred
df['pred'] = pred
df['avg_pred'] = df['pred'] / df['size']

df.drop(['percent', 'management', 'room', 'living', 'bath', 'year', 'pop',
       'sex_ratio', 'in', 'out', 'growth', 'marry_rate', 'load_archi',
       'load_house', 'M1B', 'income_rate'],
        axis=1,
        inplace=True)

df['date'] = pd.to_datetime(df['date'], format='%Y%m')

df.to_csv('web_with_pred.csv', index=False)